<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/logo-bdc.png" align="right" width="128"/>

# <span style="color:#336699">Web Land Trajectory Service (WLTS) - Base article operations</span>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    Fabiana Zioti<sup><a href="https://orcid.org/0000-0002-7305-6043"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Karine Reis Ferreira<sup><a href="https://orcid.org/0000-0003-2656-5504"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Gilberto R. Queiroz<sup><a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Alana K. Neves, Felipe Menino Carlos<sup><a href="https://orcid.org/0000-0002-3334-4315"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Felipe Carvalho de Souza<sup><a href="https://orcid.org/0000-0002-5826-1700"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Lorena Santos<sup><a href="https://orcid.org/0000-0003-2612-5859"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Rolf Simoes<sup><a href="https://orcid.org/0000-0003-0953-4132"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:brazildatacube@inpe.br">brazildatacube@inpe.br</a>
    <br/><br/>
</div>

<br/>

<div style="text-align: justify;  margin-left: 15%; margin-right: 15%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>This Jupyter Notebook is a supplement of the following paper:</b>
    <div style="margin-left: 10px; margin-right: 10px; margin-top:10px">
      <p> Zioti, F.; Ferreira, K.R.; Queiroz, G.R.; Neves, A.K.; Carlos, F.M.; Souza, F.C.; Santos, L.; Simoes, R.E; 2021. A platform for land use and land cover data integration and trajectory analysis. </p>
    </div>
</div>

In [ ]:
import os

In [ ]:
from wlts import WLTS
from lccs import LCCS

## Configurations

To run the codes presented in this document, we need to make some settings. The first one is the definition of the addresses of the **W**eb **L**and **T**rajectory **S**ervice (WLTS) and **W**eb **L**and **C**lassification **S**ystem **S**ervice (WLCSS) services.

Thus, in this example, we will use these services provided by the [Brazil Data Cube project](http://brazildatacube.org/). The addresses are defined below:


In [ ]:
#
# Web Land Trajectory Service (WLTS) URL
#
wlts_service_url = "https://brazildatacube.dpi.inpe.br/wlts/"

#
# Web Land Classification System Service (WLCSS) URL
#
lccs_service_url = "https://brazildatacube.dpi.inpe.br/lccs/"

Additionally, since we will be using the BDC services, defining an **access token** will also be necessary. This token is mandatory, and once created, gives access to all BDC services and data products.

> If you do not have an access token, you can create one through [Brazil Data Cube explorer](https://brazildatacube.dpi.inpe.br/portal/), the BDC data portal. In the portal, **create an account**, and in the **user settings**, **create your token**.

In [ ]:
bdc_access_token = "YOUR-BDC-SERVICES-TOKEN-HERE"

Finally, we will create a directory to store the data to ensure that the generated results are saved for posterior usage. In this case, we will put the data in the `analysis/data/derived_data/python-scripts` directory.

The code below creates these directories that we will use:

In [ ]:
#
# Defining the output directory
#
output_directory = "../../data/derived_data/python-scripts"

#
# Creating the output directory
#
os.makedirs(output_directory, exist_ok=True)

## Listing 1 - Retrieving LULC trajectories

This section uses the Python package `wlts.py` to access the BDC WLTS services and retrieve LULC trajectories from one point location, considering multiple collections. In this case, the following collections will be considered:

- `TerraClass Amazon`;
- `PRODES Amazon`;
- `DETER`; 
- `IBGE`;
- `MapBiomas`.


In [ ]:
service = WLTS(wlts_service_url, access_token=bdc_access_token)

**Defining the temporal period**

In [ ]:
start_date = 2000
end_date   = 2018

**Defining the collections to be consulted**

In [ ]:
collections = "prodes_amazonia_legal,deter_amazonia_legal,terraclass_amazonia,mapbiomas_amazonia-v5,ibge_cobertura_uso_terra"
collections

**Retrieving the trajectories**

In [ ]:
point_tj = service.tj(
    latitude    = -6.2829, 
    longitude   = -52.3106, 
    collections = collections,
    start_date  = start_date,
    end_date    = end_date
)

**Visualizing the trajectory as a Pandas DataFrame**

In [ ]:
point_tj.df()

**Saving the results**

In [ ]:
point_tj_df = point_tj.df()
point_tj_df.to_csv(os.path.join(output_directory, "listing1.csv"))

## Listing 3 - Harmonized trajectories

This section makes joint use of WLTS and WLCSS for retrieving LULC paths from various projects with classes harmonized between both collections. For this operation, we will use the `wlts.py` and `lccs.py` Python packages.

**Defining the services**

In [ ]:
#
# Web Land Trajectory Service (WLTS)
#
service_wlts = WLTS(url=wlts_service_url, access_token=bdc_access_token)

#
# Web Land Classification System Service (WLCSS)
#
service_lccs = LCCS(url=lccs_service_url, access_token=bdc_access_token)

**Finding the Collections classification system**

To retrieve the harmonized trajectory, it is necessary to have the classification systems of the collections and the destination classification system to which the trajectory will be mapped.
The classification system for collections can be found in the metadata provided by the WLTS:

In [ ]:
service_wlts['terraclass_amazonia-v2']

In [ ]:
service_wlts['mapbiomas_amazonia-v5']

This metadata returns the name, version identifier of the collections classification system. This information is provided directly from the WLCSS.

**Showing the available classifications system mappings**

With the information from the classification systems, it is possible to ascertain the mapping system available for the data collections that will be used in the trajectory search.

In [ ]:
service_lccs.available_mappings("TerraClass-AMZ-2")

In [ ]:
service_lccs.available_mappings("MapBiomas-5")

> Classification system Simplified-legend-TM-1 refers to the system shown in Table 3. Along with its harmonization of classes from the TerraClass and MapBiomas projects.

**Retrieving a trajectory harmonized with the `Simplified-legend-TM`**

To return the LULC trajectory in a single classification system, it is necessary to inform the desired destination system. This is done through the **target_system** parameter of the trajectory function.

In [ ]:
tj = service_wlts.tj(
    latitude      = -6.282295, 
    longitude     = -52.53655,
    collections   = 'terraclass_amazonia-v2,mapbiomas_amazonia-v5',
    target_system = 'Simplified-legend-TM-1',
    start_date    = 2004,
    end_date      = 2014
)

In [ ]:
tj.df()

**Saving the results**

In [ ]:
tj_df = tj.df()
tj_df.to_csv(os.path.join(output_directory, "listing3.csv"))